# Minor Project - HateSpeech Detection in hindi

# Imports

In [150]:
import pandas as pd
import numpy as np
import re
import emoji
import string
import csv
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn import metrics

# DataSet

In [151]:
data = pd.read_csv(r'G:/PPSU Academic/SEM  4/SSIT2930 Minor Project-II/DataSet.csv')
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [152]:
data["Post"][0]

'मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏'

# Text cleaning

In [153]:
STOPWORDS = ["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप",
             "इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी",
             "इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके",
             "उनको","उन्हीं","उन्हें","उन्हों","उस","उसके",'थी,',"उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर",
             "और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा",'कहाँ',"का","काफि","काफ़ी",
             "कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर",'हम',"किस","किसि","किसी","किसे","की",
             "कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ",
             "जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे",
             "जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि",
             "थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे",
             "निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला",
             "बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि",'सैन्\u200dय',
             'सम्\u200dमान', 'नई', 'दिल्\u200dली', 'संस्\u200dकार।',
             "यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग",
             "वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि",'श्री','स्\u200dटीफन','माध्\u200dयम', 
             "सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है",
             "हैं","हो","होता","होति","होती","होते","होना","होने"'थी,', 'वो','आ','हैं.', 'नए', 'हैं,','जाएगी', 'है।','है', 'क्यों','है,']


In [154]:
def stopword_removal(new_post):
    tokens = word_tokenize(new_post)
    stopword_remove_row = [word for word in tokens if word not in STOPWORDS]
    return stopword_remove_row

In [155]:
def emojis_removal(post):
    list1 = []
    for i in post:
        allchars = [str for str in i]
        emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
        clean_text = ' '.join([str for str in i.split() if not any(i in str for i in emoji_list)])
        list1.append(clean_text)
    return list1

In [156]:
def puncuation_removal(list):
    list2 = []
    for i in list:
        new_str = re.sub(r'[a-zA-Z0-9\n\',.#@_:…।?/|!$*-]', r'',i)
        list2.append(new_str)
    return list2

In [157]:
new_post = []
new  = " "
for row in data["Post"]:
    allchars = [str for str in row]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in row.split() if not any(i in str for i in emoji_list)])
    entity_prefixes = ['@','#', '_', '-','h']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = clean_text.replace(separator,' ')
            words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
                    new = ' '.join(words)
    
    tokens = word_tokenize(new)
    stopword_remove = [word for word in tokens if word not in STOPWORDS]
    
    print(new)

मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम
सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.
सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर :
साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।
अनलॉक-4 के लिए गाइडलाइन्स जारी 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत कंटेनमेंट जोन में कोई छूट नहीं सिनेमाहॉल अभी बंद रहेंगे 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.
चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने। चुल्लू भर मूत्र में डूब मरो गद्दारों। अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें…विक्रम शर्मा
देश में के रिकॉर्ड मामले
Exam center से निकले को सुन बा

छात्रा 11वीं क्लास में पढ़ती थी, प्रैक्टिकल परीक्षा देने के लिए स्कूल गई हुई थी, परीक्षा देकर स्कूल से लौट ही रही थी, तभी स्कूल के बाहर घात लगाए बैठे एक मुसलमान युवक ने जिसकी उम्र लगभग 25 से 26 साल बताई जा रही है, तेजी से तलवार लेकर आया, और छात्रा के गर्दन पर वार कर दिया।मुस्लिम युवक ने तलवार से वार कर हिंदू लडकी पूजा,पुत्री शंकर लाल कोतमा अनूपगढ मध्य प्रदेश की हत्या इसलिये कर दी क्योंकि उसने उसके तथाकथित लव जेहादी प्यार को नकार दिया था ,हम हिंदू इस देश मे कितने लाचार हैं। ऒर कितने सुरक्षित हैं इन जेहादियो के आगे । खून खोलता है मेरा ऐसे मै
कंगना राणावत ने मुम्बई की तुलना POK से की है, क्या अब उनके खिलाफ देशद्रोही का मुकदमा दर्ज होगा ? क्या कंगना जेल जायेंगी ? या फिर जाति और धर्म के हिसाब से उन्हें बख्श दिया जायेगा ?
क्या चालू वित्त वर्ष की पहली तिमाही में जीडीपी के खराब आंकड़ों के लिए कोरोना आपदा जिम्मेदार है?
होटल मैनेजर की आत्महत्या का मामला, पत्नी,साली समेत 4 लोगों पर केस दर्ज, जहर खाते समय बनाया था वीडियो, पत्नी-परिजनों को बताया मौत का जिम्मेदार, सिविल लाइन थाने में दर्ज हुआ है केस

ब्रेकिंग न्यूज़: रूस की सड़कों पर 500 से अधिक शेर छोड़े गए ताकि यह सुनिश्चित किया जा सके कि लोग इस महामारी के प्रकोप के दौरान घर के अंदर रह रहे हैं। व्लादिमीर पुतिन ने लोगों को घर के अंदर रहने के लिए लगभग 500 शेरों को छोड़ा।
गणित की मुश्किलों को आसान बनाने वाले प्रयागराज के इस टीचर को मिलेगा राज्य शिक्षक पुरस्कार
: के सिकंदरा इलाके में एक केमिकल फैक्ट्री में आग लगी। अब तक किसी के हताहत होने की सूचना नहीं है।
NEET/JEE और NDA के परीक्षार्थियों के लिए बिहार में पूर्व मध्य रेलवे चलाएगा 7 इंटरसिटी ट्रेनें। 4 सितंबर से 15 सितंबर के बीच चलेंगी ट्रेनें।
केंद्र सरकार ने लॉकडाउन के दौरान लोगों की नौकरी बचाने या नौकरी देने के लिए जो शानदार कदम उठाए हैं, उन कदमों के बारे में जानने के लिए इस ट्वीट के रिप्लाई में 'विश्वगुरु' लिखिए. आपको सरकार के उठाए कदमों की एक लंबी लिस्ट दिखने लगेगी.
कल, द्वारा संचालित 5,130 बसों के माध्यम से 6.37 लाख लोगों ने यात्रा की: ACS, गृह व सूचना, श्री जी
क्या एंटीवायरल दवाएं लेकर कोरोना से बचा जा सकता है?
कैलिफ़ोर्निया में सुशांत के लिए निकली ‘सत्याग्रह’ कार रैली, बहन श्वे

डब्ल्यूएचओ ने कहा है : कोरोना वायरस अब कमजोर पड़ने लगा है।
लगभग सभी राज्यों के मुख्यमंत्रियों एवं प्रशासनिक स्तर के उच्चाधिकारियों से हुई बातचीत के आधार पर मैं सभी अभ्यर्थियों एवं उनके अभिभावकों को आश्वस्त करता हूं कि उन्हें यथासंभव प्रत्येक सहायता प्रदान की जाएगी। आप सभी के उज्ज्वल भविष्य के लिए मेरी अग्रिम शुभकामनाएं!
: बिकरू कांड में Vikas Dubey के खजांची Jai Bajpai के तीनों भाइयों का सरेंडर|ABPGanga
वायरल की गयी फोटो पे लिखा है की राहुल गाँधी के प्रधान मंत्री बनने तक उधारी बंद।
झूठ बोलने में देश को अपने मित्र उद्योगपति के हाथ बेचने में देश के नागरिक को लूट कर अपने मित्रों में दान करने मैं मिसाल बनी दुनिया में भारत 6 साल में भारत को 100 साल पीछे धकेल दिया यदि 70 साल मिली होती तो भारत 29 राज्य 29 देश में बदल जाती
कफील खान का भाषण हिंसा भड़काने वाला नहीं, एकता का संदेश: HC
कांग्रेस ने की संदीप सिंह के खिलाफ जांच की मांग, BJP से संबंध होने का लगाया आरोप
शिवराज सरकार का फैसला- कलेक्टर ही कहे जाएंगे जिलाधिकारी, नहीं बदलेगा पदनाम
माना कि वो रंडी है, गंजेड़ी है, कमीनी कुतिया,,,,,,,, सब है, 

आपदा से निपटने के लिए जनपद एवं राज्य स्तर पर आपदा नियंत्रण केन्द्र की स्थापना की गई है: राहत आयुक्त, श्री संजय गोयल जी
जिसमें लिखा है, काश के कश्मीर के पत्थरबाज़ इन नवजवानों से कुछ सीखते। ।
तुमने अब तक 10 फिल्म में भी काम नहीं की है तुम्हारे पास हजारों करोड़ की प्रॉपर्टी कहां से आई इतनी शान शौकत से जिंदगी किस इनकम से कट रही है सफाई दे देश को
यूपीः बहराइच में पुलिस के हत्थे चढ़ा वाहन चोर गिरोह, 13 मोटरसाइकिल बरामद
सुशांत के परिवार पर अब किसका 'कीचड़'? पर ट्वीट कर दें अपनी राय Watch LIVE
OnePlus 8T की कथित तस्वीर लीक, मिली डिज़ाइन की झलक
मामला: डिप्रेशन को लेकर तैयार की गई थ्योरी से संतुष्ट नहीं है CBI, दिशा सालयान की मौत को लेकर भी की पूछताछ की रिपोर्ट
तेज वृद्धि के साथ भारत में 4.23 करोड़ से अधिक जांच के कुल मामलों में से 43% अकेले 3 राज्यों- महाराष्ट्र, आंध्र प्रदेश और कर्नाटक में दर्ज किए गए:
मानव अंग तस्करी हो सकती है
भारत एक लोकताँत्रिक देश है यहाँ संविधान के मुताविक सरकार जनता द्वारा पाँच साल के लिए चुनी जाती है। कोरोना महामारी की आड़ में श्री रंजन गोगोई जी ने यह कहकर भारतीय संविधा

...

सात महिला टीचर्स समेत बिहार के बीस शिक्षकों को मिलेगा राजकीय शिक्षक पुरस्कार, ये रही पूरी लिस्ट
जुत्तों से संक्रमण फैलता है? से के फैलने और लोगों को संक्रमित करने की सम्भावना बहुत कम है।
जरा गुजरात के कोरोना मामलों का भी जिक्र कर दिया करे श्रीमान... वैसे मध्यप्रदेश के तो हालत इतने अच्छे हैं की लॉकडाउन लगाना पड़ रहा है... कम से कम राजस्थान में तो ऐसा नहीं हो रहा भाईजी...
कुतिया कुछ लोग योगी होते हैं। जिनको उनके घरवाले जबरदस्ती गृहस्थ जीवन में बांधना चाहतें हैं परन्तु ऐसे लोगों के जीवन का उद्देश्य कुछ और ही होता है।और वो ये बंधन तोड़कर समाज सेवा और देश सेवा है लग जाते हैं। वैसे तो तु पप्पू की तरह अनपढ़ गंवार है परन्तु फिर भी तुम मैं एक उदाहरण के
20 लोग 197 फीट ऊपर एक घंटे तक उल्टे लटके रहे..!
जम्मू और कश्मीर में चल रहे सैन्य अभियानों का वीडियो है ।
रूस के साथ हुई 7 लाख से ज्यादा AK-47 203 राइफल की डील, 1 मिनट में दागती है 600 गोलियां via
Realme V3 हो सकता है कंपनी का सबसे सस्ता 5जी स्मार्टफोन
अंत्योदय योजना के अन्तर्गत 38440.884 मी. टन गेहूं का वितरण, 25627.256 मी. टन चावल तथा 3868.487 म

भरपेट कमा लेने के बाद जैसे N-95 मास्क बेकार घोषित हो गया। यकीन करें कमाई के बाद सेनेटाइजर, हैंडवाश और immunity booster दवायें भी बेअसर घोषित हो जाएंगी। फिर वैक्सीन से कमाने का दौर शुरू होगा। आप डर बनाये रखिये और हम बाजार बनाये रखते हैं। World Horror Organization (WHO).
दोस्ती अगर निकम्मे और नालायक से हो..... तो वो अपने साथ आपकी भी भद्द पिटवा सकता है ये बात शास्त्रों में तो लिखी ही है पर अब ये ज्ञान चीनी विचारक अपने राष्ट्रपति जिनपिंग को दे रहे..!!
जिले के पुलिसकर्मियों का सराहनीय कार्य, अजय सिंह के परिवार को पुलिसकर्मियों ने अपना एक दिन का वेतन देकर मदद की, 24 लाख 89 हजार 826 रुपए की सहायता, विभाग ने अजय सिंह की पत्नी को सौंपा, स्वाट टीम प्रभारी की कोरोना से हुई थी मौत.
स्टाम्प शुल्क कटौती से प्रोत्साहित होंगे खरीदार, बढ़ेगी घरों की डिमांड: रियल एस्टेट कंपनियां
अब चुमार में घुसपैठ की कोशिश नाकाम, भारत बोला-सैनिकों को काबू में रखे चीन
ईस कमीना को राज्यसभा से निष्कासित कर ३०७ के तहत केस कर देना चाहिए!
Gold Price Today: भारी गिरावट के बाद फिर से मजबूत हुआ सोना, बढ़ने लगी है कीमत via
1 में

भारत-चीन सीमा पर मातृभूमि की रक्षा करते हुए शहीद हुए झुंझुनूं जिले के हुकुमपुरा गाँव के वीर सपूत सूबेदार शमशेर अली खान जी की शहादत को शत शत नमन। परमपिता परमेश्वर दिवंगत आत्मा को शांति प्रदान करें व परिजनों को यह दुख सहन करने का संबल प्रदान करे।
जीबी रोड में रहने वाली एक सेक्स वर्कर ने सुनाई लॉकडाउन में अपनी ज़िंदगी की कहानी
शिक्षा नीति से जितना शिक्षक, अभिभावक जुड़े होंगे, छात्र जुड़े होंगे, उतना ही उसकी प्रासंगिकता और व्यापकता, दोनों ही बढ़ती है। देश के लाखों लोगों ने, शहर में रहने वाले, गांव में रहने वाले, शिक्षा क्षेत्र से जुड़े लोगों ने, इसके लिए अपना फीडबैक दिया था, अपने सुझाव दिए थे: PM
...और यूपी के सीएम योगी आदित्यनाथ ने प्रियंका गांधी वाड्रा के पत्र का पर लिया संज्ञान, मांगा बसों और उसके ड्राइवर का ब्यौरा। अब प्रियंका प्रवासी मजदूरों को ले जाने के लिए कांग्रेस द्वारा भेजे जाने आलू बसों की जानकारी दें
पुरानी पिक्चर नहीं है ताजी है ऐसी हरकत गृह मंत्री रहते भी की थीं पहले भी कर चुके है यह अंग्रेज के संतान
कृपया संज्ञान में लेकर आवश्यक कार्यवाही करें।
Breaking: पैंगोंग के ब्लैक टॉ

पोखरण में परमाणु क्षमताओं का प्रदर्शन हो या इसरो का मिशन मंगलयान या 104 उपग्रहों को एक साथ प्रक्षेपित करने का विश्व रिकॉर्ड या डीआरडीओ सहित अन्य संस्थानों की उपलब्धियां। भारतीय वैज्ञानिकों व प्रौद्योगिकी वाहकों ने अपनी विशेष उपलब्धियों से सम्पूर्ण विश्व में अलग पहचान बनाई है।
लिटिल मास्टर सुनील गावस्कर बोले, 'धोनी को 'ऊपर' खेलने आना चाहिए था'
महाराष्ट्र सरकार ने पेंगुइन वाले इमोजी पर प्रतिबंध लगा दिया है।
शॉकिंग! इन मूर्खों की इतनी हिम्मत की हिन्दू देवी-देवताओं की तस्वीरें सी.ए.ए के ख़िलाफ प्रदर्शन में जलाएं? सी.ए.ए का हिन्दुओं से क्या लेना देना? उनका एजेंडा कुछ और है और & के नेता इसे पाल रहे हैं|
असलहे के बल पर ऑटो चालकों से वसूली, सपा जिला अध्यक्ष के साथ पीड़ित ऑटो चालक पहुंचे एसपी ऑफिस, जबरन पैसे वसूलने की एसपी को दी शिकायत, ठेकेदार पर लगाया वसूली का आरोप।
जिस तरीके से इंडियन आर्मी आगे बढ़ रही है वो देखकर लगता है की BJP और दस साल रही तो चीन का कद नेपाल के बराबर हो
जंग' की आड़ में चीन की भुखमरी को छिपा रहे जिनपिंग, 1962 में भी ऐसे ही थे हालात via
खेद के साथ कहना पड़ता है कि " भारत "बा

बात तो पूरी सही है, पर यदि मोदी ने लिख दिया कि “ कोई धन्नासेठ किसानों की मेहनत कौड़ियों के दाम पर लूटना चाहेगा तो सरकार उसकी रक्षा करेगी” तो साले तुम ही साइड बदल के व्यापारियों के तरफ़ से चिल्लाने लगोगे की सरकार तानाशाही कर रही है! भो श्री के दोगले हो तुम लोग!!!
JDU की वर्चुअल रैली से पहले चिराग पासवान ने बुलाई LJP नेताओं की बैठक, कर सकते हैं कोई बड़ा फैसला (Report- )
दिन में तीन बार चाय पीने से कोरोना नहीं होगा।
कई न्यूज़ चैनलों ने इसी घटना की तस्वीर का इस्तेमाल करते हुए दावा किया कि यह हैदराबाद में वेटनरी डॉक्टर के साथ हुए बलात्कार और हत्या के बाद पुलिस द्वारा किये गए एनकाउंटर की वास्तविक तस्वीर है।
उत्तराखंड में तेज़ी से पांव पसार रहा कोरोना, जानिए- किस जिले का है क्या हाल?
योगी जी, अभी फिर मेरी बात कोंसी कला के थाना प्रभारी मथुरा से 3 दिन पहले घटना के बारे में FIR को लेकर हुई, जिसमें एक आदमी का मौके पर ही क़त्ल कर दिया गया था बहुत अफ़सोस की बात है अभी भी थाना प्रभारी जांच की बात कर रहा है FIR नहीं।
NCB ने रिया से दूसरे राउंड की पूछताछ शुरू की, आज शाम तक रिया की हो सकती है गिरफ्तारी


In [159]:
post = data["Post"]
new_post = emojis_removal(post)
post1 = puncuation_removal(new_post)


In [162]:
post1

['मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम',
 'सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नईनई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े ',
 'सुशांत ने जो बिजनेस डील  जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर    ',
 ' साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है',
 '  अनलॉक के लिए गाइडलाइन्स जारी   सितंबर से देशभर में मेट्रो सेवा शुरु होगी   सितंबर के बाद रैलियों और बाकी फंक्शन में  लोगों को इजाजत  कंटेनमेंट जोन में कोई छूट नहीं  सिनेमाहॉल अभी बंद रहेंगे   से वीं के छात्र  सितंबर के बाद स्कूल जा सकेंगे ',
 'चीन ने  में तर्क दिया की भारत का विपक्ष ही अजर\u200c मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करेंविक्रम शर्मा',
 'देश में  के रिकॉर्ड मामले   ',
 '   से निक

In [164]:
data_x = post1
data_y = data["Labels Set"]


In [165]:
len(data_x)

5728

In [166]:
len(data_y)

5728

In [167]:
data_dict = { 
    
    "post": data_x,
    "labels": data_y,   
}


# Cleaned DataSet

In [168]:
data_new = pd.DataFrame(data_dict)
data_new.head()

,post,labels
0,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के...,"hate,offensive"
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,सुशांत ने जो बिजनेस डील जून को की थी वो डील द...,non-hostile
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive"
4,अनलॉक के लिए गाइडलाइन्स जारी सितंबर से देश...,non-hostile


#  CountVectorizer

In [170]:
cv = CountVectorizer()

In [171]:
x=cv.fit(data_new['post'])
#y=cv.fit(data_new['labels'])
print(x.vocabulary_)
print(cv.get_feature_names())

{'बह': 1778, 'पक': 1557, 'भक': 1793, 'और': 683, 'बर': 1745, 'जय': 1082, 'सरक': 2473, 'हम': 2554, 'कम': 751, 'बढ': 1710, 'नईनई': 1471, 'रहत': 2142, 'उन': 426, 'पर': 1601, 'आर': 243, 'पड': 1579, 'जन': 1045, 'हत': 2535, 'आई': 159, 'छत': 1008, 'रत': 2069, 'अर': 95, 'नब': 1509, 'पब': 1596, 'एनय': 537, 'यह': 2019, 'कहत': 803, 'सबक': 2440, 'अध': 39, 'सच': 2407, 'नह': 1548, 'अनल': 49, 'इडल': 295, 'इन': 304, 'शभर': 2336, 'शन': 2324, 'इज': 285, 'जत': 1042, 'नम': 1513, 'अभ': 81, 'रह': 2139, 'सक': 2397, 'तर': 1307, 'अजर': 21, 'मस': 1931, 'आत': 209, 'नत': 1495, 'भर': 1816, 'मर': 1911, 'गद': 871, 'अब': 77, 'चन': 961, 'कर': 762, 'रम': 2111, 'शर': 2341, 'मल': 1920, 'कल': 779, 'हर': 2564, 'एग': 508, 'जल': 1103, 'यर': 1997, 'इल': 334, 'नजर': 1490, 'आन': 217, 'वनस': 2267, 'घर': 931, 'करन': 772, 'मन': 1889, 'रखन': 2034, 'नए': 1472, 'शहर': 2359, 'इतन': 298, 'नय': 1518, 'अपन': 56, 'जड': 1041, 'जम': 1076, 'बरस': 1763, 'जर': 1097, 'रन': 2091, 'वह': 2302, 'लड': 2185, 'जब': 1068, 'खड': 817, 'सय': 2470, 'इसल': 3

In [172]:
target_data = []


for i in data_new['labels']:
    if('non-hostile' in i):
        target_data.append(int(1))
    else:
        target_data.append(int(0))

target_data

[0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,


In [173]:
x=cv.transform(data_new['post'])
print(x.shape)
print(len(target_data))

(5728, 2630)
5728


In [174]:
dataset=x.toarray()
print(dataset)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [175]:
#optional
vectorized=pd.DataFrame(x.toarray(),columns=cv.get_feature_names())
vectorized.head()

,अक,अकड,अकबर,अकल,अकव,अख,अखब,अखल,अग,अगर,...,ड़त,ड़न,फ़टक,फ़र,फ़रम,फ़रवर,०स,३०७,५००,७त
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Split DataSet into train and test

In [183]:
X_train,X_test,Y_train,Y_test = train_test_split(vectorized,target_data,test_size=0.25, random_state=42)

In [184]:
len(X_train)

4296

In [185]:
len(X_test)

1432

In [186]:
len(Y_train)

4296

In [187]:
len(Y_test)

1432

In [188]:
set_target_data=target_data[0:4296]
len(set_target_data)

4296

In [189]:
X_train

,अक,अकड,अकबर,अकल,अकव,अख,अखब,अखल,अग,अगर,...,ड़त,ड़न,फ़टक,फ़र,फ़रम,फ़रवर,०स,३०७,५००,७त
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2742,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2721,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modal traning using Gaussian Naive Bayes

In [194]:
model = GaussianNB()
model.fit(X_train,set_target_data)

GaussianNB()

In [195]:
expected= set_target_data
predicated =model.predict(X_train)

In [196]:
print(metrics.classification_report(expected,predicated))


              precision    recall  f1-score   support

           0       0.55      1.00      0.71      2000
           1       1.00      0.30      0.46      2296

    accuracy                           0.62      4296
   macro avg       0.78      0.65      0.58      4296
weighted avg       0.79      0.62      0.58      4296



In [197]:
print(metrics.confusion_matrix(expected,predicated))

[[1999    1]
 [1617  679]]
